# 용산구_최종 순위
* 스쿨존+거주인구, 유동인구, 자동차등록 데이터 별 가중치 합계로 Top3 동 선정
* 거주인구 / 스쿨존 -> 가중치 50
* 유동인구 -> 가중치 30
* 자동차등록 -> 가중치 20

In [1]:
import pandas as pd
import numpy as np

## 용산구_스쿨존+거주인구 가중치

### 용산구_거주인구 데이터

In [2]:
pop = pd.read_csv("data/용산구_거주인구 순위.csv")
pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  16 non-null     int64  
 1   기간          16 non-null     int64  
 2   자치구         16 non-null     object 
 3   동           16 non-null     object 
 4   주차장         16 non-null     int64  
 5   mean        16 non-null     float64
 6   result      16 non-null     float64
dtypes: float64(2), int64(3), object(2)
memory usage: 1.0+ KB


In [3]:
# 데이터 정리
pop = pop.drop(['기간', '주차장', 'result'], axis=1)
pop.rename(columns = {'result' : 'mean'}, inplace = True)
pop

,Unnamed: 0,자치구,동,mean
0,3,용산구,남영동,8168.000000
1,12,용산구,보광동,16037.083333
2,11,용산구,서빙고동,13827.166667
3,6,용산구,용문동,12825.666667
4,2,용산구,용산2가동,11630.666667
5,14,용산구,원효로1동,13935.083333
6,4,용산구,원효로2동,15424.250000
7,7,용산구,이촌1동,27751.916667
8,8,용산구,이촌2동,9402.583333
9,9,용산구,이태원1동,7435.666667


### 용산구_스쿨존 데이터

In [4]:
sz = pd.read_csv("data/서울시 행정동별 어린이 보호구역 지정 통계_용산구.csv", engine='python')
sz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   자치구명    16 non-null     object
 1   행정동명    16 non-null     object
 2   현황(개소)  16 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 512.0+ bytes


In [5]:
#컬럼 이름 변경
sz.rename(columns = {'자치구명' : '자치구'}, inplace = True)
sz.rename(columns = {'행정동명' : '동'}, inplace = True)
sz.rename(columns = {'현황(개소)' : '개소'}, inplace = True)

#합계 행 삭제
sz = sz.drop(0, axis=0)

#동 이름 맞추기
sz.loc[4] = ('용산구', '원효로1동', 2)
sz.loc[5] = ('용산구', '원효로2동', 3)
sz.loc[9] = ('용산구', '이촌1동', 7)
sz.loc[10] = ('용산구', '이촌2동', 1)
sz.loc[11] = ('용산구', '이태원1동', 2)
sz.loc[12] = ('용산구', '이태원2동', 2)
sz

,자치구,동,개소
1,용산구,후암동,3
2,용산구,용산2가동,1
3,용산구,청파동,3
4,용산구,원효로1동,2
5,용산구,원효로2동,3
6,용산구,효창동,1
7,용산구,용문동,1
8,용산구,한강로동,2
9,용산구,이촌1동,7
10,용산구,이촌2동,1


### 거주인구 + 스쿨존 데이터 합치기

In [6]:
#데이터 병합
s = pd.merge(pop, sz, how='outer')
s = s.sort_values(by=['동'])

#Null 값 제거
s = s.dropna(0)
s = s.drop(['Unnamed: 0'], axis=1)

#동 이름 맞추기
s.loc[7] = ('용산구', '이촌동', 6.722942, 8.0)
s = s.drop(8, axis=0)
s.loc[9] = ('용산구', '이태원동', 5.140446, 4.0)
s = s.drop(10, axis=0)
s

,자치구,동,mean,개소
1,용산구,보광동,16037.083333,2.0
2,용산구,서빙고동,13827.166667,3.0
3,용산구,용문동,12825.666667,1.0
4,용산구,용산2가동,11630.666667,1.0
5,용산구,원효로1동,13935.083333,2.0
6,용산구,원효로2동,15424.250000,3.0
7,용산구,이촌동,6.722942,8.0
9,용산구,이태원동,5.140446,4.0
11,용산구,청파동,21255.500000,3.0
12,용산구,한강로동,14925.500000,2.0


In [7]:
s['거주인구 비'] = s['mean'] /  21255.500000
s['점수'] = s['개소'] * s['거주인구 비']
s

,자치구,동,mean,개소,거주인구 비,점수
1,용산구,보광동,16037.083333,2.0,0.754491,1.508982
2,용산구,서빙고동,13827.166667,3.0,0.650522,1.951565
3,용산구,용문동,12825.666667,1.0,0.603405,0.603405
4,용산구,용산2가동,11630.666667,1.0,0.547184,0.547184
5,용산구,원효로1동,13935.083333,2.0,0.655599,1.311198
6,용산구,원효로2동,15424.250000,3.0,0.725659,2.176978
7,용산구,이촌동,6.722942,8.0,0.000316,0.002530
9,용산구,이태원동,5.140446,4.0,0.000242,0.000967
11,용산구,청파동,21255.500000,3.0,1.000000,3.000000
12,용산구,한강로동,14925.500000,2.0,0.702195,1.404389


### 용산구_스쿨존+거주인구 가중치

In [8]:
s['스쿨존 가중치'] = s['점수'] / 3.977590 * 50
s_f = s.drop(['mean', '개소', '거주인구 비', '점수'], axis=1)
s_f

,자치구,동,스쿨존 가중치
1,용산구,보광동,18.968546
2,용산구,서빙고동,24.532009
3,용산구,용문동,7.585053
4,용산구,용산2가동,6.878334
5,용산구,원효로1동,16.482316
6,용산구,원효로2동,27.365537
7,용산구,이촌동,0.031807
9,용산구,이태원동,0.012160
11,용산구,청파동,37.711277
12,용산구,한강로동,17.653773


## 용산구_유동인구 가중치

### 용산구_유동인구 데이터

In [9]:
y = pd.read_csv("data/용산구_유동인구 순위.csv")
y

,Unnamed: 0,자치구,동,주차장,P,result
0,2,용산구,용산2가동,1684,16.340639,0.009703
1,9,용산구,이태원1동,2742,19.447155,0.007092
2,12,용산구,보광동,2033,9.928529,0.004884
3,13,용산구,청파동,5743,27.668684,0.004818
4,3,용산구,남영동,5405,22.589558,0.004179
5,5,용산구,효창동,2991,11.545432,0.003860
6,10,용산구,이태원2동,4017,11.898217,0.002962
7,4,용산구,원효로2동,5196,14.427836,0.002777
8,15,용산구,한강로동,21443,55.312881,0.002580
9,16,용산구,한남동,15669,38.308934,0.002445


In [10]:
#데이터 정리
y = y.drop(['Unnamed: 0'], axis=1)
y.rename(columns = {'result' : 'P/주차장'}, inplace = True)

#동 이름 맞추기
y.loc[1] = ('용산구', '이태원동', 6759, 31.345372, 0.010054)
y = y.drop(6, axis=0)
y.loc[13] = ('용산구', '이촌동', 12702, 22.504077, 0.003575)
y = y.drop(15, axis=0)
y

,자치구,동,주차장,P,P/주차장
0,용산구,용산2가동,1684,16.340639,0.009703
1,용산구,이태원동,6759,31.345372,0.010054
2,용산구,보광동,2033,9.928529,0.004884
3,용산구,청파동,5743,27.668684,0.004818
4,용산구,남영동,5405,22.589558,0.004179
5,용산구,효창동,2991,11.545432,0.003860
7,용산구,원효로2동,5196,14.427836,0.002777
8,용산구,한강로동,21443,55.312881,0.002580
9,용산구,한남동,15669,38.308934,0.002445
10,용산구,후암동,4561,10.938469,0.002398


### 용산구_유동인구 가중치

In [11]:
y['유동인구 가중치'] = y['P/주차장'] / 0.010054 * 30
y_f = y.drop(['주차장', 'P', 'P/주차장'], axis=1)
y_f

,자치구,동,유동인구 가중치
0,용산구,용산2가동,28.954049
1,용산구,이태원동,30.000000
2,용산구,보광동,14.572361
3,용산구,청파동,14.375801
4,용산구,남영동,12.470803
5,용산구,효창동,11.517975
7,용산구,원효로2동,8.285418
8,용산구,한강로동,7.697029
9,용산구,한남동,7.295267
10,용산구,후암동,7.156141


## 용산구_자동차등록 가중치

### 용산구_자동차등록 데이터

In [12]:
c = pd.read_csv("data/용산구_자동차등록현황 순위.csv")
c

,Unnamed: 0,자치구,동,주차장,대수,result
0,10,용산구,보광동,2033,3162.0,1.555337
1,2,용산구,용산2가동,1684,2281.0,1.354513
2,7,용산구,이촌동,12702,14534.0,1.144229
3,5,용산구,효창동,2991,2732.0,0.913407
4,8,용산구,이태원동,6759,4821.0,0.713271
5,11,용산구,청파동,5743,3084.0,0.537002
6,13,용산구,한강로동,21443,11115.0,0.518351
7,14,용산구,한남동,15669,7210.0,0.460144
8,6,용산구,용문동,3877,1727.0,0.445448
9,9,용산구,서빙고동,6527,2160.0,0.330933


In [13]:
#데이터 정리
c = c.drop(['Unnamed: 0'], axis=1)
c.rename(columns = {'result' : '대수/주차장'}, inplace = True)
c

,자치구,동,주차장,대수,대수/주차장
0,용산구,보광동,2033,3162.0,1.555337
1,용산구,용산2가동,1684,2281.0,1.354513
2,용산구,이촌동,12702,14534.0,1.144229
3,용산구,효창동,2991,2732.0,0.913407
4,용산구,이태원동,6759,4821.0,0.713271
5,용산구,청파동,5743,3084.0,0.537002
6,용산구,한강로동,21443,11115.0,0.518351
7,용산구,한남동,15669,7210.0,0.460144
8,용산구,용문동,3877,1727.0,0.445448
9,용산구,서빙고동,6527,2160.0,0.330933


### 용산구_자동차등록 가중치

In [14]:
c['자동차등록 가중치'] = c['대수/주차장'] / 1.555337 * 20
c_f = c.drop(['주차장', '대수', '대수/주차장'], axis=1)
c_f

,자치구,동,자동차등록 가중치
0,용산구,보광동,19.999999
1,용산구,용산2가동,17.417615
2,용산구,이촌동,14.713586
3,용산구,효창동,11.745453
4,용산구,이태원동,9.171918
5,용산구,청파동,6.905276
6,용산구,한강로동,6.665449
7,용산구,한남동,5.916971
8,용산구,용문동,5.727987
9,용산구,서빙고동,4.255451


## 용산구_최종 순위

In [15]:
#데이터 병합
d = pd.merge(s_f, y_f, how='outer')
d = d.sort_values(by=['동'])
d

,자치구,동,스쿨존 가중치,유동인구 가중치
13,용산구,남영동,NaN,12.470803
0,용산구,보광동,18.968546,14.572361
1,용산구,서빙고동,24.532009,5.495612
2,용산구,용문동,7.585053,6.493932
3,용산구,용산2가동,6.878334,28.954049
4,용산구,원효로1동,16.482316,5.319883
5,용산구,원효로2동,27.365537,8.285418
6,용산구,이촌동,0.031807,10.667396
7,용산구,이태원동,0.012160,30.000000
8,용산구,청파동,37.711277,14.375801


In [16]:
df = pd.merge(d, c_f, how='outer')
df = df.sort_values(by=['동'])
df

,자치구,동,스쿨존 가중치,유동인구 가중치,자동차등록 가중치
0,용산구,남영동,NaN,12.470803,0.582876
1,용산구,보광동,18.968546,14.572361,19.999999
2,용산구,서빙고동,24.532009,5.495612,4.255451
3,용산구,용문동,7.585053,6.493932,5.727987
4,용산구,용산2가동,6.878334,28.954049,17.417615
5,용산구,원효로1동,16.482316,5.319883,3.269390
6,용산구,원효로2동,27.365537,8.285418,1.960025
7,용산구,이촌동,0.031807,10.667396,14.713586
8,용산구,이태원동,0.012160,30.000000,9.171918
9,용산구,청파동,37.711277,14.375801,6.905276


In [17]:
#Null 값 제거
df = df.dropna(0)
df

,자치구,동,스쿨존 가중치,유동인구 가중치,자동차등록 가중치
1,용산구,보광동,18.968546,14.572361,19.999999
2,용산구,서빙고동,24.532009,5.495612,4.255451
3,용산구,용문동,7.585053,6.493932,5.727987
4,용산구,용산2가동,6.878334,28.954049,17.417615
5,용산구,원효로1동,16.482316,5.319883,3.269390
6,용산구,원효로2동,27.365537,8.285418,1.960025
7,용산구,이촌동,0.031807,10.667396,14.713586
8,용산구,이태원동,0.012160,30.000000,9.171918
9,용산구,청파동,37.711277,14.375801,6.905276
10,용산구,한강로동,17.653773,7.697029,6.665449


In [18]:
df['sum'] = df['스쿨존 가중치'] + df['유동인구 가중치'] + df['자동차등록 가중치']
df = df.sort_values(by=['sum'], ascending=False)
df

<ipython-input-18-29a6a125b9ed>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sum'] = df['스쿨존 가중치'] + df['유동인구 가중치'] + df['자동차등록 가중치']


,자치구,동,스쿨존 가중치,유동인구 가중치,자동차등록 가중치,sum
11,용산구,한남동,50.000001,7.295267,5.916971,63.212239
9,용산구,청파동,37.711277,14.375801,6.905276,58.992355
1,용산구,보광동,18.968546,14.572361,19.999999,53.540906
4,용산구,용산2가동,6.878334,28.954049,17.417615,53.249999
8,용산구,이태원동,0.012160,30.000000,9.171918,39.184078
6,용산구,원효로2동,27.365537,8.285418,1.960025,37.610980
2,용산구,서빙고동,24.532009,5.495612,4.255451,34.283073
10,용산구,한강로동,17.653773,7.697029,6.665449,32.016251
12,용산구,효창동,5.633543,11.517975,11.745453,28.896971
7,용산구,이촌동,0.031807,10.667396,14.713586,25.412790


In [19]:
#csv 파일로 저장
df.to_csv("용산구_최종순위.csv")

In [20]:
df_1 = pd.read_csv("용산구_최종순위.csv")
df_1

,Unnamed: 0,자치구,동,스쿨존 가중치,유동인구 가중치,자동차등록 가중치,sum
0,11,용산구,한남동,50.000001,7.295267,5.916971,63.212239
1,9,용산구,청파동,37.711277,14.375801,6.905276,58.992355
2,1,용산구,보광동,18.968546,14.572361,19.999999,53.540906
3,4,용산구,용산2가동,6.878334,28.954049,17.417615,53.249999
4,8,용산구,이태원동,0.012160,30.000000,9.171918,39.184078
5,6,용산구,원효로2동,27.365537,8.285418,1.960025,37.610980
6,2,용산구,서빙고동,24.532009,5.495612,4.255451,34.283073
7,10,용산구,한강로동,17.653773,7.697029,6.665449,32.016251
8,12,용산구,효창동,5.633543,11.517975,11.745453,28.896971
9,7,용산구,이촌동,0.031807,10.667396,14.713586,25.412790
